In [1]:
import secrets
secrets.token_hex(16)

'611db7bc16337ab41d6e55198385a38e'

# Import

In [1]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
port=5432
def get_db_connection():
    """Create and return a database connection"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST', 'localhost'),
            database=os.getenv('DB_NAME', 'jadwapla_db'),
            user=os.getenv('DB_USER', 'postgres'),
            password=os.getenv('DB_PASSWORD', 'jadwaplan'),
            port=os.getenv('DB_PORT', '5432')
        )
        print("✅ Database connected successfully!")
        return conn
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None

# Test connection
conn = get_db_connection()

✅ Database connected successfully!


In [2]:
def run_query(query, params=None, fetch=True):
    """Run a SQL query and return results"""
    try:
        with conn.cursor() as cur:
            cur.execute(query, params or ())
            if fetch:
                if query.strip().upper().startswith('SELECT'):
                    columns = [desc[0] for desc in cur.description]
                    results = cur.fetchall()
                    return columns, results
                else:
                    conn.commit()
                    return f"Query executed successfully. Rows affected: {cur.rowcount}"
            else:
                conn.commit()
                return "Query executed successfully"
    except Exception as e:
        conn.rollback()
        return f"Error: {e}"

def query_to_dataframe(query, params=None):
    """Run query and return as pandas DataFrame"""
    columns, results = run_query(query, params)
    return pd.DataFrame(results, columns=columns)

def show_tables():
    """Show all tables in the database"""
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
    """
    return query_to_dataframe(query)

In [6]:
query_to_dataframe('select * from buz_export_plan').columns

Index(['bplan_id', 'client_avatar', 'full_name', 'client_gender',
       'client_profile', 'client_experiences', 'client_partners',
       'client_expenses', 'client_employed', 'side_business',
       'business_profile', 'buz_staff', 'buz_resource', 'business_premises',
       'market_analysis', 'buz_suppliers', 'buz_production',
       'enhance_production', 'buz_distribution', 'customer_support',
       'requested_fund', 'feasibility', 'complete_client_profile',
       'complete_business_profile', 'complete_business_premises',
       'complete_market_analysis', 'complete_competitors',
       'complete_operations_plan', 'complete_requested_fund',
       'complete_feasibility', 'source_of_funding', 'financial_history',
       'mission', 'vision'],
      dtype='object')

# delete bplan

In [11]:
q='select * from bplan'
query_to_dataframe(q)

,bplan_id,name,email,industry,buz_sector,buz_subsector,buz_currency,creation_date,status,completion,complete_client_profile,complete_business_profile,complete_business_premises,complete_market_analysis,complete_competitors,complete_operations_plan,complete_requested_fund,complete_feasibility,user_id
0,7,aaaa,boss@email.com,,,,USD,2025-10-24 12:51:06.647351,1,30,True,True,False,False,False,False,False,False,1.0
1,4,yousseftest,boss@email.com,3,Telecommunications,,EURO,2025-09-02 12:44:59.175836,1,100,True,True,True,True,True,True,True,True,1.0
2,5,youssef Test,boss@email.com,2,Construction Materials,,USD,2025-09-28 16:16:00.980568,1,50,True,True,True,False,False,False,False,True,NaN


In [12]:
def cleanup_bplan_data(keep_ids=[4, 5, 7]):
    """
    Delete all rows from tables with bplan_id where bplan_id is NOT in the keep list
    """
    print(f"🧹 Starting cleanup - keeping only bplan_id in {keep_ids}")

    # Get all tables with bplan_id column
    bplan_tables_query = """
    SELECT table_name
    FROM information_schema.columns
    WHERE table_schema = 'public'
    AND column_name = 'bplan_id'
    ORDER BY table_name;
    """

    columns, tables_result = run_query(bplan_tables_query)
    tables_to_clean = [row[0] for row in tables_result]

    print(f"📋 Found {len(tables_to_clean)} tables with bplan_id: {tables_to_clean}")

    deletion_summary = {}

    for table in tables_to_clean:
        try:
            # Count records before deletion
            count_before_query = f"SELECT COUNT(*) FROM public.{table}"
            _, count_before_result = run_query(count_before_query)
            count_before = count_before_result[0][0]

            # Count records to keep
            count_keep_query = f"SELECT COUNT(*) FROM public.{table} WHERE bplan_id IN %s"
            _, count_keep_result = run_query(count_keep_query, (tuple(keep_ids),))
            count_keep = count_keep_result[0][0]

            # Delete records NOT in keep list
            delete_query = f"DELETE FROM public.{table} WHERE bplan_id NOT IN %s"
            result = run_query(delete_query, (tuple(keep_ids),), fetch=False)

            # Count records after deletion
            count_after_query = f"SELECT COUNT(*) FROM public.{table}"
            _, count_after_result = run_query(count_after_query)
            count_after = count_after_result[0][0]

            deletion_summary[table] = {
                'before': count_before,
                'keep': count_keep,
                'after': count_after,
                'deleted': count_before - count_after
            }

            print(f"✅ {table}: {count_before} → {count_after} records (deleted {count_before - count_after})")

        except Exception as e:
            print(f"❌ Error cleaning {table}: {e}")
            deletion_summary[table] = {'error': str(e)}

    return deletion_summary

# Run the cleanup
print("🚀 Starting database cleanup...")
summary = cleanup_bplan_data([4, 5, 7])

# Display summary
print("\n📊 CLEANUP SUMMARY:")
print("=" * 50)
for table, stats in summary.items():
    if 'error' in stats:
        print(f"{table}: ❌ ERROR - {stats['error']}")
    else:
        print(f"{table}: {stats['before']} → {stats['after']} records (deleted {stats['deleted']})")

🚀 Starting database cleanup...
🧹 Starting cleanup - keeping only bplan_id in [4, 5, 7]
📋 Found 34 tables with bplan_id: ['bplan', 'buz_competitor', 'buz_distribution', 'buz_expenses', 'buz_export_plan', 'buz_feasibility', 'buz_feasibility_operating_expenses', 'buz_financial_history', 'buz_fund_details', 'buz_fund_items', 'buz_info', 'buz_mkt_analysis', 'buz_operation_plan', 'buz_oresource', 'buz_other_resource', 'buz_preferences', 'buz_premise', 'buz_premises_doc', 'buz_premises_photo', 'buz_product_services', 'buz_production', 'buz_products', 'buz_resource', 'buz_staff', 'buz_supplier', 'buz_suppliers', 'buz_suppliers_products_info', 'buz_yearly_commission', 'client_info', 'employed', 'expenses', 'experiences', 'partners', 'side_business']
✅ bplan: 3 → 3 records (deleted 0)
✅ buz_competitor: 20 → 3 records (deleted 17)
✅ buz_distribution: 4 → 4 records (deleted 0)
✅ buz_expenses: 0 → 0 records (deleted 0)
✅ buz_export_plan: 20 → 3 records (deleted 17)
✅ buz_feasibility: 20 → 3 records

In [13]:
def get_all_distinct_bplan_ids():
    """
    Retrieve all distinct bplan_id values across all tables containing a bplan_id column.
    """
    print("🔍 Fetching all distinct bplan_id values across tables...")

    # Find all tables with a bplan_id column
    bplan_tables_query = """
    SELECT table_name
    FROM information_schema.columns
    WHERE table_schema = 'public'
    AND column_name = 'bplan_id'
    ORDER BY table_name;
    """
    _, tables_result = run_query(bplan_tables_query)
    tables = [row[0] for row in tables_result]

    print(f"📋 Found {len(tables)} tables with bplan_id: {tables}")

    all_bplan_ids = set()
    table_counts = {}

    # Loop through each table and collect distinct bplan_id values
    for table in tables:
        try:
            query = f"SELECT DISTINCT bplan_id FROM public.{table}"
            _, result = run_query(query)
            bplan_ids = {row[0] for row in result if row[0] is not None}
            all_bplan_ids.update(bplan_ids)
            table_counts[table] = len(bplan_ids)
            print(f"✅ {table}: {len(bplan_ids)} distinct bplan_id(s) found")

        except Exception as e:
            print(f"❌ Error reading {table}: {e}")
            table_counts[table] = f"Error: {e}"

    print("\n📊 SUMMARY OF DISTINCT bplan_id PER TABLE")
    print("=" * 50)
    for table, count in table_counts.items():
        print(f"{table}: {count}")

    print("\n🔢 TOTAL UNIQUE bplan_id VALUES FOUND:", len(all_bplan_ids))
    print(f"🧩 Combined bplan_id values: {sorted(all_bplan_ids)}")

    return all_bplan_ids, table_counts


In [14]:
print("🚀 Checking distinct bplan_id values...")
all_bplans, table_summary = get_all_distinct_bplan_ids()


🚀 Checking distinct bplan_id values...
🔍 Fetching all distinct bplan_id values across tables...
📋 Found 34 tables with bplan_id: ['bplan', 'buz_competitor', 'buz_distribution', 'buz_expenses', 'buz_export_plan', 'buz_feasibility', 'buz_feasibility_operating_expenses', 'buz_financial_history', 'buz_fund_details', 'buz_fund_items', 'buz_info', 'buz_mkt_analysis', 'buz_operation_plan', 'buz_oresource', 'buz_other_resource', 'buz_preferences', 'buz_premise', 'buz_premises_doc', 'buz_premises_photo', 'buz_product_services', 'buz_production', 'buz_products', 'buz_resource', 'buz_staff', 'buz_supplier', 'buz_suppliers', 'buz_suppliers_products_info', 'buz_yearly_commission', 'client_info', 'employed', 'expenses', 'experiences', 'partners', 'side_business']
✅ bplan: 3 distinct bplan_id(s) found
✅ buz_competitor: 3 distinct bplan_id(s) found
✅ buz_distribution: 1 distinct bplan_id(s) found
✅ buz_expenses: 0 distinct bplan_id(s) found
✅ buz_export_plan: 3 distinct bplan_id(s) found
✅ buz_feasibi

TypeError: '<' not supported between instances of 'int' and 'str'

In [17]:
def get_all_unique_bplan_ids():
    """
    Return a sorted list of all distinct bplan_id values across all tables
    containing a bplan_id column.
    Handles mixed int/str types gracefully.
    """
    print("🔍 Collecting all distinct bplan_id values across tables...")

    # Find all tables with a bplan_id column
    bplan_tables_query = """
    SELECT table_name
    FROM information_schema.columns
    WHERE table_schema = 'public'
    AND column_name = 'bplan_id'
    ORDER BY table_name;
    """
    _, tables_result = run_query(bplan_tables_query)
    tables = [row[0] for row in tables_result]

    all_bplan_ids = set()

    for table in tables:
        try:
            query = f"SELECT DISTINCT bplan_id FROM public.{table}"
            _, result = run_query(query)
            for row in result:
                val = row[0]
                if val is None:
                    continue
                # Try to cast to int safely
                try:
                    val = int(val)
                except (ValueError, TypeError):
                    val = str(val)
                all_bplan_ids.add(val)
        except Exception as e:
            print(f"❌ Error reading {table}: {e}")

    # Sort with consistent type handling
    all_bplan_ids = sorted(all_bplan_ids, key=lambda x: (isinstance(x, str), x))

    print(f"\n🧩 Unique bplan_id values across all tables: {all_bplan_ids}")
    print(f"🔢 Total distinct bplan_id: {len(all_bplan_ids)}")

    return all_bplan_ids


# API

In [30]:
OPENAI_API_KEY = "sk-proj-VwTYgACukwJ7Erp5unKIT3BlbkFJwClnWksCSVGV7qdYmPoE"

In [ ]:
def get_api_content_buz_product_services(client, query_model, bplan_id, lang='en'):
    conn = None
    try:
        db_params = config()
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()

        # Step 1: Fetch product/services info
        sql = """
        SELECT
            'Product/Service name: ' || product_service_name ||
            ', Description: ' || COALESCE(product_service_description, 'No description')
        AS api_content
        FROM public.buz_product_services
        WHERE bplan_id = %s;
        """
        cur.execute(sql, (bplan_id,))
        rows = cur.fetchall()

        if not rows:
            chatgpt_product_services = ''
        else:
            # Prepare concatenated content
            results = [row[0] for row in rows]
            content_input = " and ".join(results)

            # --- Build multilingual prompt
            base_prompt = """Write a professional description of the products and services offered by
                    this business in about 100 words using a third-person perspective.
                    Combine and rewrite the information below into a coherent, polished paragraph.
                    You MUST ONLY use the information provided. DO NOT invent, add, or create anything that are not explicitly mentioned in the provided content."""
            if lang == "ar":
                base_prompt += " Do it in Arabic."


            query_messages = [
                {"role": "user", "content": base_prompt},
                {"role": "user", "content": content_input}
            ]

            # Step 2: Generate description with GPT
            result = client.chat.completions.create(
                model=query_model,
                messages=query_messages
            )

            chatgpt_product_services = result.choices[0].message.content.replace("'", "''")

        # Step 3: Update export plan
        sql_update = """
        UPDATE public.buz_export_plan
        SET buz_product_services = %s
        WHERE bplan_id = %s;
        """
        cur.execute(sql_update, (chatgpt_product_services, bplan_id))

        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"API PRODUCT/SERVICES ERROR: {error}")
        return (False, error)
    finally:
        if conn is not None:
            conn.commit()
            conn.close()

    return (True, "")

In [31]:
def test_business_profile_generation(bplan_id, lang='en'):
    """
    Test function for business profile generation in notebook
    """
    from openai import OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    query_model = "gpt-4o-mini"

    conn = get_db_connection()
    cur = conn.cursor()

    try:
        # Test the SQL query first
        sql = "SELECT ' brand name is ' || buz_name || ' established since ' || TO_CHAR(buz_est_date, 'DD/MM/YYYY')  || '. Located in ' || buz_address || '. business model ' ||  buz_model as api_content FROM public.buz_info WHERE bplan_id = {};".format(bplan_id)
        print(f"🔍 SQL Query: {sql}")

        cur.execute(sql)
        print(f"✅ Found {cur.rowcount} business info records")

        if cur.rowcount == 0:
            print("❌ No business info found")
            return None
        else:
            results = cur.fetchall()
            print(f"📊 Raw business info: {results[0][0]}")

            base_prompt = """You are a professional business writer. Create a compelling company overview paragraph (approximately 120-180 words) that:

            1. Presents the company's brand, establishment, and business model
            2. Uses formal third-person business language
            3. Highlights unique value proposition and market positioning
            4. Avoids list-like structures and repetition
            5. Creates a cohesive narrative about the business
            6. Output a single, continuous paragraph only.
            You MUST ONLY use the information provided. DO NOT invent, add, or create anything that are not explicitly mentioned in the provided content."""

            if lang == "ar":
                base_prompt += " Do it in Arabic."

            query_messages = [
                {"role": "user", "content": base_prompt},
                {"role": "user", "content": results[0][0]}
            ]

            print("🚀 Calling OpenAI API for business profile...")
            result_business_profile = client.chat.completions.create(model=query_model, messages=query_messages)
            chatgpt_business_profile = result_business_profile.choices[0].message.content

            print("✅ Generated business profile:")
            print("=" * 50)
            print(chatgpt_business_profile)
            print("=" * 50)

            # Update the database
            update_sql = "UPDATE public.buz_export_plan SET business_profile = %s WHERE bplan_id = %s;"
            cur.execute(update_sql, (chatgpt_business_profile, bplan_id))
            conn.commit()
            print("✅ Business profile saved to database")

            return {
                "business_profile": chatgpt_business_profile,
                "raw_input": results[0][0]
            }

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None
    finally:
        cur.close()
        conn.close()

# Run the test
test_result = test_business_profile_generation(4, 'en')

✅ Database connected successfully!
🔍 SQL Query: SELECT ' brand name is ' || buz_name || ' established since ' || TO_CHAR(buz_est_date, 'DD/MM/YYYY')  || '. Located in ' || buz_address || '. business model ' ||  buz_model as api_content FROM public.buz_info WHERE bplan_id = 4;
✅ Found 1 business info records
📊 Raw business info:  brand name is yousseftest established since 01/10/2025. Located in Tripoli. business model [1]
🚀 Calling OpenAI API for business profile...
✅ Generated business profile:
YoussefTest, established on October 1, 2025, in Tripoli, represents a pioneering force in its industry, distinguished by its innovative approach and commitment to quality. The company operates on a robust business model that prioritizes customer satisfaction and adaptability to market demands. Focusing on delivering exceptional testing solutions, YoussefTest leverages cutting-edge technology and expert insights to enhance performance and reliability in various applications. Its unique value pro

# testing

In [3]:
show_tables()

,table_name
0,Users
1,bplan
2,buz_competitor
3,buz_distribution
4,buz_expenses
5,buz_export_plan
6,buz_feasibility
7,buz_feasibility_operating_expenses
8,buz_financial_history
9,buz_fund_details


In [7]:
query_to_dataframe('select * from buz_mkt_analysis where bplan_id=4')

,bplan_id,segment_id,segment_name,business_model,segment_percentage,market_channel,age_min,age_max,income_min,income_max,...,location,preferences,industry,company_size,show_age_range,show_income_range,show_gender_percentage,show_education,show_occupation,show_life_stage
0,4,18,fadi,B2C,69.00,"[3, 6, 7, 8]",10,50,500,8000,...,Lille,swimming,Agriculture,None,on,off,off,on,off,on
1,4,10,Kaza,B2C,30.00,"[2, 8]",25,30,500,8000,...,Paris,,Agriculture,None,on,off,on,on,off,on
2,4,4,plan,B2B,1.00,[],20,80,500,8000,...,Paris,big companies,Technology,None,off,off,off,off,off,off
